# Прогноз добычи нефти. Подготовка массивов для обучения
Задача - спрогнозировать добычу нефти, исходя из данных по короткому начальному интервалу добычи, а также геологической и технической информации по скважине.

Возможны два подхода. "Параметрический" основан на предположении, что добыча падает по определенному закону (например, по закону Арпса, справедливость которого была подтверждена в первой части проекта), и модель должна подобрать параметры в формуле. "Непараметрический" не предполагает какую-то заданную форму кривой. Во втором случае модель можно будет применять только на временном интервале, использованном при обучении. Параметрический метод дает модель, не ограниченную временным интервалом обучения. Сначала для простоты пробую непараметрический метод.

Если наблюдать за добычей достаточно долго, то можно определить коэффициенты уравнения без машинного обучения. Но в данном случае для прогноза будет использован короткий период.

В этой версии для обучения будет использован одинаковый период времени для всех скважин, равный самому короткой продолжительности наблюдения для всех скважин.

Целевая переменная - общий объем, добытый за весь этот период. Признаки - зависящие от времени данные (дебит, давление в скважине) за короткий начальный период (train_period) и независящие от времени геологические и технические данные, что потребует построения модели с двумя входами.

In [33]:
import pandas as pd

In [34]:
well_data = pd.read_csv('well_data_oil.csv', index_col=0)
well_data

,inverse_spacing,Initial_Pressure_Estimate_ksi,Dew_Point_Pressure_kpsi,Bubble_Point_Pressure_kpsi,Reservoir_Temperature_degFe2,Sandface_Temp_degFe2,Static_Wellhead_Temp_degFe2,Stages_e2,Net_Pay_kft,TVD_kft,...,Initial_GOR_Mscf/bbl,Clusters_e3,Porosity_e1,Clusters_per_Stage_e1,Total_Proppant_lbs_e8,Total_Fluid_Bbls_e6,Oil_Specific_Gravity,Water_Saturation,Oil_Saturation,Pressure_Source_Tubing_Pressure
CARDINAL,1.428571,5.900,2.440,2.440,2.28,2.280,0.70,0.49,0.067,8.759000,...,0.75684,0.735,0.52,1.5,0.265768,0.542527,0.876161,0.360,0.640,NaN
CROW,0.769231,5.500,1.764,1.764,2.21,2.200,0.70,0.69,0.063,8.420200,...,0.51307,1.035,0.52,1.5,0.390200,0.575663,0.813218,0.321,0.679,NaN
EAGLE,0.000000,5.000,2.122,2.122,2.35,2.340,0.60,0.50,0.074,7.890810,...,0.55790,0.450,0.63,0.9,0.252674,0.577003,0.847508,0.270,0.730,NaN
FALCON,1.000000,5.700,1.370,1.370,2.25,2.250,0.60,0.64,0.066,7.501876,...,0.27500,0.960,0.63,1.5,0.359710,0.559202,0.851896,0.270,0.730,NaN
HAWK,1.000000,5.650,1.370,1.370,2.25,2.250,0.60,0.68,0.067,7.516675,...,0.27500,1.020,0.63,1.5,0.380648,0.596128,0.851896,0.271,0.729,NaN
JAY,1.428571,5.700,2.459,2.459,2.28,2.279,0.70,0.48,0.067,8.790710,...,0.76384,0.432,0.52,0.9,0.259860,0.519038,0.876161,0.361,0.602,NaN
KITE,0.714286,5.000,3.064,3.064,2.38,2.350,0.65,0.34,0.056,8.846830,...,1.03470,0.306,0.50,0.9,0.185548,0.360202,0.808202,0.321,0.679,NaN
LARK,1.250000,5.700,2.455,2.455,2.28,2.280,0.70,0.50,0.068,8.779860,...,0.76228,0.450,0.52,0.9,0.270246,0.535169,0.876161,0.363,0.637,NaN
OSPREY,0.714286,5.400,1.211,1.211,2.25,2.250,0.70,0.28,0.078,7.638600,...,0.33572,0.252,0.63,0.9,0.135145,0.275579,0.838320,0.260,0.740,NaN
SPARROW,0.666667,5.164,2.100,2.100,2.25,2.250,0.70,0.49,0.062,8.536400,...,0.62840,0.441,0.52,0.9,0.275507,0.533414,0.876161,0.332,0.668,NaN


In [35]:
production_data = pd.read_csv('production_by_period_oil.csv', index_col=0)
production_data

,period,day,Gas Volume (MMscf),flowrate,Gas Lift Inj Volume (MMscf),Casing_Pressure_ksia,Tubing_Pressure_ksia,Active_Pressure_ksia,Line_Pressure_ksia,Calculated_Sandface_Pressure_ksia,water_thousand_bbl
CARDINAL,CARDINAL_0.0_119.0,0.0,0.24546,321.67,0.0,0.908000,1.820000,1.820000,0.059586,5.142505,0.66333
CARDINAL,CARDINAL_0.0_119.0,1.0,0.24935,337.89,0.0,0.912000,1.767000,1.767000,0.059586,5.089437,0.69000
CARDINAL,CARDINAL_0.0_119.0,2.0,0.25026,336.47,0.0,0.900000,1.747000,1.747000,0.059586,5.049800,0.64667
CARDINAL,CARDINAL_0.0_119.0,3.0,0.25158,353.89,0.0,0.897000,1.725000,1.725000,0.059586,4.995304,0.61000
CARDINAL,CARDINAL_0.0_119.0,4.0,0.24229,336.78,0.0,0.895000,1.711000,1.711000,0.059586,4.985308,0.59167
...,...,...,...,...,...,...,...,...,...,...,...
SWIFT,SWIFT_896.0_1245.0,231.0,0.03237,61.00,0.0,0.086496,0.119640,0.086496,0.014696,1.844417,0.04100
SWIFT,SWIFT_896.0_1245.0,232.0,0.03100,53.00,0.0,0.084821,0.097029,0.084821,0.014696,1.795906,0.00000
SWIFT,SWIFT_896.0_1245.0,233.0,0.03000,28.00,0.0,0.086696,0.117196,0.086696,0.014696,1.851028,0.02300
SWIFT,SWIFT_896.0_1245.0,234.0,0.01400,37.00,0.0,0.085596,0.103846,0.085596,0.014696,1.867198,0.05000


In [36]:
start = production_data\
    .groupby('period')\
    .agg({'day': 'min'})\
    .rename(columns={'day': 'start'})
start

,start
period,
CARDINAL_0.0_119.0,0.0
CARDINAL_119.0_486.0,0.0
CARDINAL_486.0_781.0,0.0
CROW_24.0_216.0,0.0
EAGLE_0.0_205.0,1.0
EAGLE_205.0_1483.0,0.0
JAY_0.0_132.0,0.0
JAY_132.0_480.0,0.0
JAY_480.0_782.0,0.0


Начало разное, поэтому выполняю сдвиг.

In [37]:
production_data = production_data.join(start, on='period')

In [38]:
production_data

,period,day,Gas Volume (MMscf),flowrate,Gas Lift Inj Volume (MMscf),Casing_Pressure_ksia,Tubing_Pressure_ksia,Active_Pressure_ksia,Line_Pressure_ksia,Calculated_Sandface_Pressure_ksia,water_thousand_bbl,start
CARDINAL,CARDINAL_0.0_119.0,0.0,0.24546,321.67,0.0,0.908000,1.820000,1.820000,0.059586,5.142505,0.66333,0.0
CARDINAL,CARDINAL_0.0_119.0,1.0,0.24935,337.89,0.0,0.912000,1.767000,1.767000,0.059586,5.089437,0.69000,0.0
CARDINAL,CARDINAL_0.0_119.0,2.0,0.25026,336.47,0.0,0.900000,1.747000,1.747000,0.059586,5.049800,0.64667,0.0
CARDINAL,CARDINAL_0.0_119.0,3.0,0.25158,353.89,0.0,0.897000,1.725000,1.725000,0.059586,4.995304,0.61000,0.0
CARDINAL,CARDINAL_0.0_119.0,4.0,0.24229,336.78,0.0,0.895000,1.711000,1.711000,0.059586,4.985308,0.59167,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
SWIFT,SWIFT_896.0_1245.0,231.0,0.03237,61.00,0.0,0.086496,0.119640,0.086496,0.014696,1.844417,0.04100,0.0
SWIFT,SWIFT_896.0_1245.0,232.0,0.03100,53.00,0.0,0.084821,0.097029,0.084821,0.014696,1.795906,0.00000,0.0
SWIFT,SWIFT_896.0_1245.0,233.0,0.03000,28.00,0.0,0.086696,0.117196,0.086696,0.014696,1.851028,0.02300,0.0
SWIFT,SWIFT_896.0_1245.0,234.0,0.01400,37.00,0.0,0.085596,0.103846,0.085596,0.014696,1.867198,0.05000,0.0


In [39]:
production_data['day'] = production_data.day - production_data.start

In [40]:
production_data.drop(columns=['start'], inplace=True)

Проверяю:

In [41]:
production_data.groupby('period').agg({'day': 'min'})

,day
period,
CARDINAL_0.0_119.0,0.0
CARDINAL_119.0_486.0,0.0
CARDINAL_486.0_781.0,0.0
CROW_24.0_216.0,0.0
EAGLE_0.0_205.0,0.0
EAGLE_205.0_1483.0,0.0
JAY_0.0_132.0,0.0
JAY_132.0_480.0,0.0
JAY_480.0_782.0,0.0


Добавляю периоды в данные по скважинам

In [42]:
well_data = production_data[['period']].drop_duplicates().join(well_data)

In [43]:
well_data

,period,inverse_spacing,Initial_Pressure_Estimate_ksi,Dew_Point_Pressure_kpsi,Bubble_Point_Pressure_kpsi,Reservoir_Temperature_degFe2,Sandface_Temp_degFe2,Static_Wellhead_Temp_degFe2,Stages_e2,Net_Pay_kft,...,Initial_GOR_Mscf/bbl,Clusters_e3,Porosity_e1,Clusters_per_Stage_e1,Total_Proppant_lbs_e8,Total_Fluid_Bbls_e6,Oil_Specific_Gravity,Water_Saturation,Oil_Saturation,Pressure_Source_Tubing_Pressure
CARDINAL,CARDINAL_0.0_119.0,1.428571,5.900,2.440,2.440,2.28,2.280,0.70,0.49,0.067,...,0.75684,0.735,0.52,1.5,0.265768,0.542527,0.876161,0.360,0.640,NaN
CARDINAL,CARDINAL_119.0_486.0,1.428571,5.900,2.440,2.440,2.28,2.280,0.70,0.49,0.067,...,0.75684,0.735,0.52,1.5,0.265768,0.542527,0.876161,0.360,0.640,NaN
CARDINAL,CARDINAL_486.0_781.0,1.428571,5.900,2.440,2.440,2.28,2.280,0.70,0.49,0.067,...,0.75684,0.735,0.52,1.5,0.265768,0.542527,0.876161,0.360,0.640,NaN
CROW,CROW_24.0_216.0,0.769231,5.500,1.764,1.764,2.21,2.200,0.70,0.69,0.063,...,0.51307,1.035,0.52,1.5,0.390200,0.575663,0.813218,0.321,0.679,NaN
EAGLE,EAGLE_0.0_205.0,0.000000,5.000,2.122,2.122,2.35,2.340,0.60,0.50,0.074,...,0.55790,0.450,0.63,0.9,0.252674,0.577003,0.847508,0.270,0.730,NaN
EAGLE,EAGLE_205.0_1483.0,0.000000,5.000,2.122,2.122,2.35,2.340,0.60,0.50,0.074,...,0.55790,0.450,0.63,0.9,0.252674,0.577003,0.847508,0.270,0.730,NaN
JAY,JAY_0.0_132.0,1.428571,5.700,2.459,2.459,2.28,2.279,0.70,0.48,0.067,...,0.76384,0.432,0.52,0.9,0.259860,0.519038,0.876161,0.361,0.602,NaN
JAY,JAY_132.0_480.0,1.428571,5.700,2.459,2.459,2.28,2.279,0.70,0.48,0.067,...,0.76384,0.432,0.52,0.9,0.259860,0.519038,0.876161,0.361,0.602,NaN
JAY,JAY_480.0_782.0,1.428571,5.700,2.459,2.459,2.28,2.279,0.70,0.48,0.067,...,0.76384,0.432,0.52,0.9,0.259860,0.519038,0.876161,0.361,0.602,NaN
KITE,KITE_111.0_1066.0,0.714286,5.000,3.064,3.064,2.38,2.350,0.65,0.34,0.056,...,1.03470,0.306,0.50,0.9,0.185548,0.360202,0.808202,0.321,0.679,NaN


Выборка очень маленькая, поэтому для окончательной проверки буду использовать только одну скважину с наибольшим периодом наблюдения.

In [44]:
production_data.groupby('period').agg({'day': 'max'}).sort_values(by='day', ascending=False).iloc[0, :]

day    1163.0
Name: EAGLE_205.0_1483.0, dtype: float64

In [45]:
test = production_data.query('period == "EAGLE_205.0_1483.0"')
test

,period,day,Gas Volume (MMscf),flowrate,Gas Lift Inj Volume (MMscf),Casing_Pressure_ksia,Tubing_Pressure_ksia,Active_Pressure_ksia,Line_Pressure_ksia,Calculated_Sandface_Pressure_ksia,water_thousand_bbl
EAGLE,EAGLE_205.0_1483.0,0.0,0.15700,316.730,0.000,0.261669,0.400000,0.400000,0.014696,3.052517,0.19700
EAGLE,EAGLE_205.0_1483.0,1.0,0.15400,393.412,0.000,0.262491,0.250000,0.250000,0.014696,2.730096,0.08000
EAGLE,EAGLE_205.0_1483.0,2.0,0.15100,324.598,0.000,0.263314,0.380000,0.380000,0.014696,2.853773,0.06800
EAGLE,EAGLE_205.0_1483.0,3.0,0.14900,390.811,0.000,0.264137,0.280000,0.280000,0.014696,2.777323,0.07100
EAGLE,EAGLE_205.0_1483.0,4.0,0.14800,414.277,0.000,0.264960,0.280000,0.280000,0.014696,2.827907,0.09100
...,...,...,...,...,...,...,...,...,...,...,...
EAGLE,EAGLE_205.0_1483.0,1159.0,0.05635,112.910,0.339,0.689696,0.094696,0.094696,0.085197,0.965118,0.06500
EAGLE,EAGLE_205.0_1483.0,1160.0,0.07729,126.250,0.365,0.699696,0.104696,0.104696,0.104611,0.941650,0.06800
EAGLE,EAGLE_205.0_1483.0,1161.0,0.07472,122.090,0.392,0.680696,0.094696,0.094696,0.107219,0.938363,0.09000
EAGLE,EAGLE_205.0_1483.0,1162.0,0.08539,182.500,0.418,0.694696,0.114696,0.114696,0.107219,1.088944,0.10100


In [46]:
train = production_data.query('period != "EAGLE_205.0_1483.0"')
train

,period,day,Gas Volume (MMscf),flowrate,Gas Lift Inj Volume (MMscf),Casing_Pressure_ksia,Tubing_Pressure_ksia,Active_Pressure_ksia,Line_Pressure_ksia,Calculated_Sandface_Pressure_ksia,water_thousand_bbl
CARDINAL,CARDINAL_0.0_119.0,0.0,0.24546,321.67,0.0,0.908000,1.820000,1.820000,0.059586,5.142505,0.66333
CARDINAL,CARDINAL_0.0_119.0,1.0,0.24935,337.89,0.0,0.912000,1.767000,1.767000,0.059586,5.089437,0.69000
CARDINAL,CARDINAL_0.0_119.0,2.0,0.25026,336.47,0.0,0.900000,1.747000,1.747000,0.059586,5.049800,0.64667
CARDINAL,CARDINAL_0.0_119.0,3.0,0.25158,353.89,0.0,0.897000,1.725000,1.725000,0.059586,4.995304,0.61000
CARDINAL,CARDINAL_0.0_119.0,4.0,0.24229,336.78,0.0,0.895000,1.711000,1.711000,0.059586,4.985308,0.59167
...,...,...,...,...,...,...,...,...,...,...,...
SWIFT,SWIFT_896.0_1245.0,231.0,0.03237,61.00,0.0,0.086496,0.119640,0.086496,0.014696,1.844417,0.04100
SWIFT,SWIFT_896.0_1245.0,232.0,0.03100,53.00,0.0,0.084821,0.097029,0.084821,0.014696,1.795906,0.00000
SWIFT,SWIFT_896.0_1245.0,233.0,0.03000,28.00,0.0,0.086696,0.117196,0.086696,0.014696,1.851028,0.02300
SWIFT,SWIFT_896.0_1245.0,234.0,0.01400,37.00,0.0,0.085596,0.103846,0.085596,0.014696,1.867198,0.05000


In [47]:
well_data_test = well_data.query('period == "EAGLE_205.0_1483.0"')
well_data_test

,period,inverse_spacing,Initial_Pressure_Estimate_ksi,Dew_Point_Pressure_kpsi,Bubble_Point_Pressure_kpsi,Reservoir_Temperature_degFe2,Sandface_Temp_degFe2,Static_Wellhead_Temp_degFe2,Stages_e2,Net_Pay_kft,...,Initial_GOR_Mscf/bbl,Clusters_e3,Porosity_e1,Clusters_per_Stage_e1,Total_Proppant_lbs_e8,Total_Fluid_Bbls_e6,Oil_Specific_Gravity,Water_Saturation,Oil_Saturation,Pressure_Source_Tubing_Pressure
EAGLE,EAGLE_205.0_1483.0,0.0,5.0,2.122,2.122,2.35,2.34,0.6,0.5,0.074,...,0.5579,0.45,0.63,0.9,0.252674,0.577003,0.847508,0.27,0.73,NaN


In [48]:
well_data_train = well_data.query('period != "EAGLE_205.0_1483.0"')
well_data_train

,period,inverse_spacing,Initial_Pressure_Estimate_ksi,Dew_Point_Pressure_kpsi,Bubble_Point_Pressure_kpsi,Reservoir_Temperature_degFe2,Sandface_Temp_degFe2,Static_Wellhead_Temp_degFe2,Stages_e2,Net_Pay_kft,...,Initial_GOR_Mscf/bbl,Clusters_e3,Porosity_e1,Clusters_per_Stage_e1,Total_Proppant_lbs_e8,Total_Fluid_Bbls_e6,Oil_Specific_Gravity,Water_Saturation,Oil_Saturation,Pressure_Source_Tubing_Pressure
CARDINAL,CARDINAL_0.0_119.0,1.428571,5.900,2.440,2.440,2.28,2.280,0.70,0.49,0.067,...,0.75684,0.735,0.52,1.5,0.265768,0.542527,0.876161,0.360,0.640,NaN
CARDINAL,CARDINAL_119.0_486.0,1.428571,5.900,2.440,2.440,2.28,2.280,0.70,0.49,0.067,...,0.75684,0.735,0.52,1.5,0.265768,0.542527,0.876161,0.360,0.640,NaN
CARDINAL,CARDINAL_486.0_781.0,1.428571,5.900,2.440,2.440,2.28,2.280,0.70,0.49,0.067,...,0.75684,0.735,0.52,1.5,0.265768,0.542527,0.876161,0.360,0.640,NaN
CROW,CROW_24.0_216.0,0.769231,5.500,1.764,1.764,2.21,2.200,0.70,0.69,0.063,...,0.51307,1.035,0.52,1.5,0.390200,0.575663,0.813218,0.321,0.679,NaN
EAGLE,EAGLE_0.0_205.0,0.000000,5.000,2.122,2.122,2.35,2.340,0.60,0.50,0.074,...,0.55790,0.450,0.63,0.9,0.252674,0.577003,0.847508,0.270,0.730,NaN
JAY,JAY_0.0_132.0,1.428571,5.700,2.459,2.459,2.28,2.279,0.70,0.48,0.067,...,0.76384,0.432,0.52,0.9,0.259860,0.519038,0.876161,0.361,0.602,NaN
JAY,JAY_132.0_480.0,1.428571,5.700,2.459,2.459,2.28,2.279,0.70,0.48,0.067,...,0.76384,0.432,0.52,0.9,0.259860,0.519038,0.876161,0.361,0.602,NaN
JAY,JAY_480.0_782.0,1.428571,5.700,2.459,2.459,2.28,2.279,0.70,0.48,0.067,...,0.76384,0.432,0.52,0.9,0.259860,0.519038,0.876161,0.361,0.602,NaN
KITE,KITE_111.0_1066.0,0.714286,5.000,3.064,3.064,2.38,2.350,0.65,0.34,0.056,...,1.03470,0.306,0.50,0.9,0.185548,0.360202,0.808202,0.321,0.679,NaN
LARK,LARK_122.0_438.0,1.250000,5.700,2.455,2.455,2.28,2.280,0.70,0.50,0.068,...,0.76228,0.450,0.52,0.9,0.270246,0.535169,0.876161,0.363,0.637,NaN


Минимальная продолжительность наблюдения.

In [49]:
min_observation_period = production_data.groupby('period').agg({'day': 'max'}).min()
min_observation_period[0]

101.0

In [51]:
train.reset_index(inplace=True)

In [52]:
train

,index,period,day,Gas Volume (MMscf),flowrate,Gas Lift Inj Volume (MMscf),Casing_Pressure_ksia,Tubing_Pressure_ksia,Active_Pressure_ksia,Line_Pressure_ksia,Calculated_Sandface_Pressure_ksia,water_thousand_bbl
0,CARDINAL,CARDINAL_0.0_119.0,0.0,0.24546,321.67,0.0,0.908000,1.820000,1.820000,0.059586,5.142505,0.66333
1,CARDINAL,CARDINAL_0.0_119.0,1.0,0.24935,337.89,0.0,0.912000,1.767000,1.767000,0.059586,5.089437,0.69000
2,CARDINAL,CARDINAL_0.0_119.0,2.0,0.25026,336.47,0.0,0.900000,1.747000,1.747000,0.059586,5.049800,0.64667
3,CARDINAL,CARDINAL_0.0_119.0,3.0,0.25158,353.89,0.0,0.897000,1.725000,1.725000,0.059586,4.995304,0.61000
4,CARDINAL,CARDINAL_0.0_119.0,4.0,0.24229,336.78,0.0,0.895000,1.711000,1.711000,0.059586,4.985308,0.59167
...,...,...,...,...,...,...,...,...,...,...,...,...
7707,SWIFT,SWIFT_896.0_1245.0,231.0,0.03237,61.00,0.0,0.086496,0.119640,0.086496,0.014696,1.844417,0.04100
7708,SWIFT,SWIFT_896.0_1245.0,232.0,0.03100,53.00,0.0,0.084821,0.097029,0.084821,0.014696,1.795906,0.00000
7709,SWIFT,SWIFT_896.0_1245.0,233.0,0.03000,28.00,0.0,0.086696,0.117196,0.086696,0.014696,1.851028,0.02300
7710,SWIFT,SWIFT_896.0_1245.0,234.0,0.01400,37.00,0.0,0.085596,0.103846,0.085596,0.014696,1.867198,0.05000


Отбираю значение целевой переменной за минимальный период наблюдения, чтобы у всех скважин время наблюдения было одинаковым.

In [28]:
y_train = train\
    .loc[train.day <= min_observation_period[0], ['period', 'flowrate']]\
    .groupby('period')\
    .agg({'flowrate': 'sum'})\
    .rename(columns={'flowrate': 'total_production'})
y_train

,total_production
period,
CARDINAL_0.0_119.0,32417.2400
CARDINAL_119.0_486.0,23956.8870
CARDINAL_486.0_781.0,8730.8000
CROW_24.0_216.0,60222.0000
EAGLE_0.0_205.0,81460.8840
JAY_0.0_132.0,27991.7800
JAY_132.0_480.0,16254.6258
JAY_480.0_782.0,5073.9700
KITE_111.0_1066.0,15051.4400


In [29]:
train_periods = [5, 10, 20, 30]

In [30]:
train_period = 30

In [53]:
X_train = train.loc[train.day <= train_period, :]\
    .drop(columns=['index', 'flowrate'])
X_train

,period,day,Gas Volume (MMscf),Gas Lift Inj Volume (MMscf),Casing_Pressure_ksia,Tubing_Pressure_ksia,Active_Pressure_ksia,Line_Pressure_ksia,Calculated_Sandface_Pressure_ksia,water_thousand_bbl
0,CARDINAL_0.0_119.0,0.0,0.24546,0.0,0.908,1.820,1.820,0.059586,5.142505,0.66333
1,CARDINAL_0.0_119.0,1.0,0.24935,0.0,0.912,1.767,1.767,0.059586,5.089437,0.69000
2,CARDINAL_0.0_119.0,2.0,0.25026,0.0,0.900,1.747,1.747,0.059586,5.049800,0.64667
3,CARDINAL_0.0_119.0,3.0,0.25158,0.0,0.897,1.725,1.725,0.059586,4.995304,0.61000
4,CARDINAL_0.0_119.0,4.0,0.24229,0.0,0.895,1.711,1.711,0.059586,4.985308,0.59167
...,...,...,...,...,...,...,...,...,...,...
7502,SWIFT_896.0_1245.0,26.0,0.05800,0.0,0.085,0.232,0.085,0.014696,1.853477,0.06000
7503,SWIFT_896.0_1245.0,27.0,0.05900,0.0,0.085,0.228,0.085,0.014696,1.852330,0.05800
7504,SWIFT_896.0_1245.0,28.0,0.06000,0.0,0.086,0.229,0.086,0.014696,1.866795,0.06600
7505,SWIFT_896.0_1245.0,29.0,0.05900,0.0,0.086,0.226,0.086,0.014696,1.849376,0.05800


Работа не закончена